Lecture 10.

In [ ]:
import torch
import torch.nn
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    # 인풋 채널 1, 아웃풋 채널 10, 필터 크기 5x5
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(320, 10) # 320 -> 10

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1)
    x = self.fc(x)
    return F.log_softmax(x)

Exercise 10-1.

In [ ]:
import torch
from torch import nn
import torch.nn
import torch.nn.functional as F

class Basic(nn.Module):

  def __init__(self):
    super(Basic, self).__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)
    self.conv4 = nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1)
    self.mp = nn.MaxPool2d(kernel_size = 2, padding = 2)
    self.fc1 = nn.Linear(20736, 256)
    self.fc2 = nn.Linear(256, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = F.relu(self.mp(self.conv2(x)))
    x = F.relu(self.mp(self.conv3(x)))
    x = F.relu(self.mp(self.conv4(x)))
    x = x.view(in_size, -1)
    x = self.fc1(x)
    x = self.fc2(x)
    return F.log_softmax(x)

model = Basic()
from torchsummary import summary
summary(model, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
         MaxPool2d-2           [-1, 32, 65, 65]               0
            Conv2d-3           [-1, 64, 65, 65]          18,496
         MaxPool2d-4           [-1, 64, 33, 33]               0
            Conv2d-5          [-1, 128, 33, 33]          73,856
         MaxPool2d-6          [-1, 128, 17, 17]               0
            Conv2d-7          [-1, 256, 17, 17]         295,168
         MaxPool2d-8            [-1, 256, 9, 9]               0
            Linear-9                  [-1, 256]       5,308,672
           Linear-10                   [-1, 10]           2,570
Total params: 5,699,658
Trainable params: 5,699,658
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 9.70
Params size (MB): 21.74
Estima

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Exercise 11-1. GoogLeNet

In [ ]:
import torch
import torch.nn as nn
        
class Inception(nn.Module): # padding은 공식 이용해서 계산, stride는 주어짐, 'S'는 'same', 'V'는 'valid'로 간주
    
    def __init__(self, common_in_ch, o_c1, o_c2a, i_c2, o_c2b, o_c3a, i_c3, o_c3b, o_c4): 
        super(Inception, self).__init__()
        
        branch1 = []
        branch1 += [nn.Conv2d(in_channels = common_in_ch, out_channels = o_c1, kernel_size = 1, stride = 1, padding = 0),
                    nn.ReLU(True)]
        
        branch2 = []
        branch2 += [nn.Conv2d(in_channels = common_in_ch, out_channels = o_c2a, kernel_size = 1, stride = 1, padding = 0),
                    nn.ReLU(True),
                    nn.Conv2d(in_channels = i_c2, out_channels = o_c2b, kernel_size = 3, stride = 1, padding = 1),
                    nn.ReLU(True)]
        
        branch3 = []
        branch3 += [nn.Conv2d(in_channels = common_in_ch, out_channels = o_c3a, kernel_size = 1, stride = 1, padding = 0),
                    nn.ReLU(True),
                    nn.Conv2d(in_channels = i_c3, out_channels = o_c3b, kernel_size = 5, stride = 1, padding = 2),
                    nn.ReLU(True)]
        
        branch4 = []
        branch4 += [nn.MaxPool2d(kernel_size = 3, stride = 1, padding = 1),
                    nn.Conv2d(in_channels = common_in_ch, out_channels = o_c4, kernel_size = 1, padding = 0),
                    nn.ReLU(True)]
        
        self.layer1 = nn.Sequential(*branch1)
        self.layer2 = nn.Sequential(*branch2)
        self.layer3 = nn.Sequential(*branch3)
        self.layer4 = nn.Sequential(*branch4)
        
    def forward(self, x):
        
        x1 = self.layer1(x)
        x2 = self.layer2(x)
        x3 = self.layer3(x)
        x4 = self.layer4(x)
        
        out = torch.cat((x1, x2, x3, x4), dim = 1) # (batchsize, channel, width, height)
        
        return out

class GoogleNet(nn.Module):
    
    def __init__(self):
        super(GoogleNet, self).__init__()
        
        layer1 = []
        layer1 += [nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, stride = 2, padding = 3), 
                   nn.ReLU(True),
                   nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
                   nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 1, stride = 1, padding = 0), # (V)로 표시되서 valid로 간주
                   nn.ReLU(True),
                   nn.Conv2d(in_channels = 64, out_channels = 192, kernel_size = 3, stride = 1, padding = 1),
                   nn.ReLU(True),
                   Inception(192, 64, 96, 96, 128, 16, 16, 32, 32),
                   Inception(256, 128, 128, 128, 192, 32, 32, 96, 64),
                   nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
                   Inception(480, 192, 96, 96, 208, 16, 16, 48, 64),
                   Inception(512, 160, 112, 112, 224, 24, 24, 64, 64)]
        
        layer2 = []
        layer2 += [Inception(512, 128, 128, 128, 256, 24, 24, 64, 64),
                   Inception(512, 112, 144, 144, 288, 32, 32, 64, 64),
                   Inception(528, 256, 160, 160, 320, 32, 32, 128, 128)]
        
        layer3 = []
        layer3 += [nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),
                   Inception(832, 256, 160, 160, 320, 32, 32, 128, 128),
                   Inception(832, 384, 192, 192, 384, 48, 48, 128, 128),
                   nn.AdaptiveAvgPool2d((7,7)),
                   nn.Dropout(0.4)]
        
        self.layer1 = nn.Sequential(*layer1)
        self.layer2 = nn.Sequential(*layer2)
        self.layer3 = nn.Sequential(*layer3)
        
        self.conv = nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 1, stride = 1, padding = 1)
        
        self.dense1a = nn.Linear(in_features = 65664, out_features = 132096) # 이것들은 모두 keras의 summary를 통해서 알아냈음
        self.dense1b = nn.Linear(in_features = 106624, out_features = 132096)
        
        self.dense2 = nn.Linear(in_features = 132096, out_features = 1049600)
        
        self.dense3 = nn.Linear(in_features = 1024, out_features = 1000)
        
        self.avgpool = nn.AdaptiveAvgPool2d((5,5)) # AdaptiveAvgPool2d은 이 형태로만, stride와 padding 안들어감
        
    def forward(self, x):
        
        x = self.layer1(x)
        
        aux0 = self.avgpool(x)
        aux0 = self.conv(aux0)
        aux0 = self.dense1a(aux0)
        aux0 = self.dense2(aux0)
        
        x = self.layer2(x)
        
        aux1 = self.avgpool(x)
        aux1 = self.conv(aux1)
        aux1 = self.dense1b(aux1)
        aux1 = self.dense2(aux1)
        
        x = self.layer3(x)
        x = x.view(x.size(0),-1)
        x =  self.dense3(x)
        
        return x, aux0, aux1

from torchsummary import summary
model = GoogleNet()
summary(model, (3, 224, 224))

Exercis 11-2. ResNet

In [8]:
import torch
import torch.nn as nn

class View(nn.Module):
    
    def __init__(self, *shape): 
        super(View, self).__init__() 
        self.shape = shape
        
    def forward(self, x):
        return x.view(x.shape[0], *self.shape) 

class Residual_Block(nn.Module):  
    
    def __init__(self, n_ch): 
        super(Residual_Block, self).__init__() 
        layers = []
        layers += [nn.BatchNorm2d(num_features=n_ch),
                  nn.ReLU(inplace=True), 
                  nn.Conv2d(in_channels=n_ch, out_channels=n_ch, kernel_size=3, stride=1, padding=1, bias=False),
                  nn.BatchNorm2d(num_features=n_ch),
                  nn.ReLU(inplace=True),
                  nn.Conv2d(in_channels=n_ch, out_channels=n_ch, kernel_size=3, stride=1, padding=1, bias=False)]
        self.layers = nn.Sequential(*layers)
        
    def forward(self,x):
        out = self.layers(x)
        return x + out
    
class ResNet(nn.Module):
    
    def __init__(self):
        super(ResNet, self).__init__()
        
        layers = []
        layers += [nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3), 
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1), 
                   Residual_Block(n_ch=64),
                   Residual_Block(n_ch=64),
                   nn.BatchNorm2d(64), 
                   nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=1),
                   Residual_Block(n_ch=256),
                   Residual_Block(n_ch=256), 
                   nn.AdaptiveAvgPool2d((1,1)), 
                   View(-1), 
                   nn.Linear(in_features=256, out_features=10)]
                      
        self.layers = nn.Sequential(*layers)
        
    def forward(self,x):
        return self.layers(x)
    
if __name__ == '__main__': 
    model = ResNet()
    summary(model, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 14, 14]           3,200
         MaxPool2d-2             [-1, 64, 7, 7]               0
       BatchNorm2d-3             [-1, 64, 7, 7]             128
              ReLU-4             [-1, 64, 7, 7]               0
            Conv2d-5             [-1, 64, 7, 7]          36,864
       BatchNorm2d-6             [-1, 64, 7, 7]             128
              ReLU-7             [-1, 64, 7, 7]               0
            Conv2d-8             [-1, 64, 7, 7]          36,864
    Residual_Block-9             [-1, 64, 7, 7]               0
      BatchNorm2d-10             [-1, 64, 7, 7]             128
             ReLU-11             [-1, 64, 7, 7]               0
           Conv2d-12             [-1, 64, 7, 7]          36,864
      BatchNorm2d-13             [-1, 64, 7, 7]             128
             ReLU-14             [-1, 6

Exercise 11-3. DenseNet

In [ ]:
import torch
import torch.nn as nn

class Bottleneck(nn.Module):

  def __init__(self, in_ch, growth_rate):
    super(Bottleneck, self).__init__()
    self.bn1 = nn.BatchNorm2d(in_ch)
    self.conv1 = nn.Conv2d(in_ch, growth_rate*4, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm2d(growth_rate*4)
    self.conv2 = nn.Conv2d(growth_rate*4, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)
    self.relu = nn.ReLU(True)

  def forward(self, x):
    x1 = self.bn1(x)
    x1 = self.relu(x1)
    x1 = self.conv1(x1)
    x1 = self.bn2(x1)
    x1 = self.relu(x1)
    x1 = self.conv2(x1)
    x = torch.cat((x, x1), dim=1)
    return x

class DenseBlock(nn.Module):

  def __init__(self, n_bottleneck, in_ch, growth_rate):
    super(DenseBlock, self).__init__()

    for i in range(n_bottleneck):
      # i번째 DenseBlock을 생성하고 거기에 Bottleneck의 속성을 차례대로 부여한다.
      setattr(self, 'DenseBlock_{}'.format(i), Bottleneck(in_ch+i*growth_rate, growth_rate))

    # forward 함수에서 n_bottleneck를 쓰기 위함
    self.n_bottleneck = n_bottleneck

  def forward(self, x):

    for i in range(self.n_bottleneck):
      # setattr을 통해 만들어진 DenseBlock들을 찾아오고, 평소처럼 레이어 x에 연결된다.
      x = getattr(self, 'DenseBlock_{}'.format(i))(x)
    return x

class Transition(nn.Module):

  def __init__(self, in_ch):
    super(Transition, self).__init__()
    num_ch = int(in_ch*0.5) # 논문에서 제시한 세타 값 = 0.5
    self.bn = nn.BatchNorm2d(in_ch)
    self.relu = nn.ReLU(True)
    self.conv = nn.Conv2d(in_ch, num_ch, kernel_size=3, padding=1, bias=False)
    self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self,x):
    x = self.bn(x)
    x = self.relu(x)
    x = self.conv(x)
    x = self.avgpool(x)
    return x

class DenseNet(nn.Module): # DenseNet-121

  def __init__(self):
    super(DenseNet, self).__init__()
    self.conv = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    self.dense1 = DenseBlock(6, 64, 32) # growth rate는 논문에서 항상 32로 고정됨
    self.trans1 = Transition(256) # 최종 in_ch+i*growth_rate -> 64+6*32 = 256
    self.dense2 = DenseBlock(12, 128, 32) # Transition의 conv 거치면 채널 수 절반으로(세타=0.5 영향). 256/2 = 128
    self.trans2 = Transition(512) # 128+12*32 = 512
    self.dense3 = DenseBlock(24, 256, 32) # 512/2 = 256
    self.trans3 = Transition(1024) # 256+24*32 = 1024
    self.dense4 = DenseBlock(16, 512, 32)
    self.avgpool = nn.AdaptiveAvgPool2d((7,7))
    self.linear = nn.Linear(50176, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = self.conv(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.dense1(x)
    x = self.trans1(x)
    x = self.dense2(x)
    x = self.trans2(x)
    x = self.dense3(x)
    x = self.trans3(x)
    x = self.dense4(x)
    x = self.avgpool(x)
    x = x.view(in_size, -1)
    x = self.linear(x)

model = DenseNet()
from torchsummary import summary
summary(model, (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
       BatchNorm2d-5           [-1, 64, 56, 56]             128
              ReLU-6           [-1, 64, 56, 56]               0
            Conv2d-7          [-1, 128, 56, 56]           8,192
       BatchNorm2d-8          [-1, 128, 56, 56]             256
              ReLU-9          [-1, 128, 56, 56]               0
           Conv2d-10           [-1, 32, 56, 56]          36,864
       Bottleneck-11           [-1, 96, 56, 56]               0
      BatchNorm2d-12           [-1, 96, 56, 56]             192
             ReLU-13           [-1, 96, 56, 56]               0
           Conv2d-14          [-1, 128,